### file 읽어오기

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_community.document_loaders import TextLoader
path = r'data/medicine.txt'

# 1. loader 객체 생성
loader = TextLoader(path, encoding='utf-8')
# 2. loader를 이용해서 파일 읽어옴
docs = loader.load()

In [20]:
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(SQLiteCache(".cache_prompt.sqlite"))

COLLECTION_NAME = "medicine_docs"
PERSIST_DIRECTORY = "vector_store/chroma/medicine_db2"
EMBEDDING_MODEL_NAME = 'text-embedding-3-small'

# Text Loading
loader = TextLoader(path, encoding='utf-8')

# 문서 load and split
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

docs = loader.load_and_split(splitter)

# Vector Store 생성

embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# Retriever 생성 - "mmr" 방식
# MMR 방식 조회
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs = {
        'k':5,
        'fetch_k':10
    }
)



In [22]:
query = '두통인데 약 추천해줘'
retriever.invoke(query)  

[Document(metadata={'source': 'data/medicine.txt'}, page_content='두두엔액\n(주)퍼슨\n이 약은 습진, 피부염, 땀띠, 가려움, 벌레물린데, 두드러기에 사용합니다.\n1일 수회(여러 차례) 환부(질환 부위)에 적당량을 바릅니다.\n이 약에 과민증 환자, 30개월 이하의 유아, 피부감염증(피부결핵, 매독, 칸디다증, 백선, 대상포진, 단순포진, 수두, 종두증, 옴, 사면발이),\xa0막 천공이 있는 습진성 외이도염, 궤양(베체트병 제외), 제2도 심재성 이상의 화상 및 동상, 입주위피부염, 보통여드름, 주사 환자는 이 약을 사용하지 마십시오.피부 감염을 수반하는 습진,\xa0상처부위, 민감한 피부에 사용하지 마십시오.이 약을 사용하기 전에\xa0임부 또는 임신하고 있을 가능성이 있는 여성 및 수유부, 소아, 고령자, 알레르기 체질,\xa0습윤(습기 참)이나 짓무름, 염증이 심한 환자는 의사 또는 약사와 상의하십시오.정해진 용법과 용량을 잘 지키십시오.어린이에게 투여할 경우 보호자의 지도 감독하에 투여하십시오.이 약은 외용으로만 사용하고 내복 또는 흡입하지 마십시오.눈 또는 눈 주위, 점막 등에는 사용하지 않아야 하며 이 약이 눈에 들어가지 않도록 주의하고, 만일 눈에 들어간 경우 즉시 물로 씻어내십시오.약액이 묻은 손이 눈 등 점막에 닿으면 자극이 있으므로 손에 묻은 약액은 잘 씻어내십시오.의사의 감독 없이 밀봉붕대법을 사용하지 마십시오.기저귀 등은 밀봉붕대법과 같은 작용을 나타낼 수 있으므로 주의하십시오.화장이나 면도 후 등 치료 이외의 목적으로 사용하지 마십시오.사용 전에 잘 흔들어 사용하십시오.염색한 머리에 사용하면 탈색될 수 있으므로 주의하십시오.\n습기와 빛을 피해 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.화기 근처에 두지 마십시오.'),
 Document(metadata={'source': 'data/medicine.txt'}, page_content='복합판시놀액\n동성제약

In [25]:
from langchain_core.output_parsers import StrOutputParser

# Prompt Template 생성
messages = [
            ("ai", """
            너는 유능한 약사야. context 안에 있는 내용에 한 해서
            사용자가 입력한 증상에 맞는 약을 2개만 선택해서 알려줘.
            복용이 간단하고 부작용이 적은 약을 우선적으로 추천해서 알려줘.
            약을 복용하기 전에 반드시 알아야 할 사항은 꼭 알려줘.
            증상이 악화될 경우 대처방법에 대해서 알려줘.
            약에 따른 복용법에 대해서도 간단히 언급해줘.
            주의사항 요약해서 알려줘. 
            위의 내용을 표로 정리해서 알려줘.       
            {context}")"""),
            ("human", "{question}")
    ]

prompt_template = ChatPromptTemplate(messages)

# 모델
model = ChatOpenAI(model='gpt-4o-mini')

#output parser
parser = StrOutputParser()

# Chain 구성 retriever (관련문서 조회) -> prompt_template -> model -> outputparser
chain = {'context':retriever, 'question':RunnablePassthrough()} | prompt_template | model | parser
    # RunnablePassThrough -> 이전 question 부름

data = chain.invoke('두통인데 약 추천해줘')

print(data)

두통에 대한 추천 약물은 다음과 같습니다:

| 약물명           | 용도                          | 복용법                     | 주의사항 및 부작용                                                   |
|------------------|-------------------------------|---------------------------|---------------------------------------------------------------------|
| 씨트리드정      | 위통, 속쓰림, 체함, 구역에 사용 | 성인: 1회 2정, 1일 2회 (4정 초과 금지) | 고혈압, 신장/간 질환 환자, 15세 미만, 고령자, 임부는 의사 상담 필요. |
| 진트로이정       | 소화기 기능 이상에 사용       | 성인: 1회 100-200mg, 1일 3회 식전 복용 | 임부 및 수유부는 의사 상담 필요.                                    |

### 복용 전 알아야 할 사항:
- 모든 약물은 의사 또는 약사와 상담 후 복용하는 것이 안전합니다.
- 특히 고령자나 기존 질환이 있는 경우 추가적인 주의가 필요합니다.

### 증상 악화 시 대처 방법:
- 두통이 지속되거나 심해질 경우 즉시 의사와 상담해야 합니다.
- 두통이 심할 경우 휴식과 수분 섭취도 중요합니다.

### 주의사항 요약:
- 약물 복용 시 권장 용량을 초과하지 않도록 주의하세요.
- 부작용이 발생할 경우 즉시 사용을 중단하고 의사에게 상담하세요.

두통이 계속되거나 증상이 심해지면 반드시 전문가와 상담하시기 바랍니다.
